# Task 2


### 2.1 Conditional Generative Adversarial Nets (cGAN)

First we need to load the data and prepare it

In [ ]:
import numpy as np
import math
import random
import os
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, utils
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import preprocessing
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from interpret.glassbox import ExplainableBoostingClassifier


In [ ]:
df = pd.read_csv('UNSW_NB15_testing-set.csv')
df.head()


,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,...,1,1,0,0,0,1,1,0,Normal,0
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,...,1,2,0,0,0,1,6,0,Normal,0
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,...,1,3,0,0,0,2,6,0,Normal,0
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,1,3,1,1,0,2,1,0,Normal,0
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,...,1,40,0,0,0,2,39,0,Normal,0


In [ ]:
for elem in df.columns:
    if df[elem].dtype == 'object':
        print(elem)

proto
service
state
attack_cat


In [ ]:
df.attack_cat.unique()

array(['Normal', 'Backdoor', 'Analysis', 'Fuzzers', 'Shellcode',
       'Reconnaissance', 'Exploits', 'DoS', 'Worms', 'Generic'],
      dtype=object)

Dropping categorical features

In [ ]:
names = df['attack_cat']
df = df.drop(["proto" ,"service" ,"state", 'attack_cat'], axis = 1)
df.shape

(175341, 41)

Encoding attack categories

In [ ]:
le = preprocessing.LabelEncoder()
classes = pd.DataFrame(le.fit_transform(names))
df = pd.concat([df,pd.DataFrame(le.fit_transform(names), columns = ['attack_cat'])], axis = 1)
# classes = le.classes_
df.attack_cat.unique()
classes

,0
0,6
1,6
2,6
3,6
4,6
...,...
175336,5
175337,8
175338,5
175339,5


Detecting those features that have constant values and removing them

In [ ]:
constant_cols = []
for elem in df.columns:
    group = df.groupby(df[elem].tolist(), as_index=False).size().drop(['index'], axis = 1)
    dis = group / len(df[elem])
    if any(0.99< np.array(dis).reshape(1,-1)[0]):
        constant_cols.append(elem)
constant_cols

[]

In [ ]:
labels = df.attack_cat
df = df.drop(constant_cols + ['attack_cat'], axis = 1)
# l = df.attack_cat


Scaling the data using Min Max scaler

In [ ]:
scaler = MinMaxScaler() 
df = pd.DataFrame(scaler.fit_transform(df))


defining a data loader class for ***dataloader*** method

In [ ]:
class Loader(Dataset):
    
    def __init__(self, data , label):
        self.data = data
        self.label = label

    def __getitem__(self, index):
        return self.data[index], self.label[index]
    def __len__(self):
        return len(self.data)

In [ ]:
input_data = np.array([(m,i) for m, i in zip(np.array(df), np.array(labels))])
dataloader = DataLoader(Loader(np.array(df), np.array(labels)), batch_size=64,shuffle=True)
dataloader

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


Defining the generator and discriminator classes and corresponding loss function for each of them.


In [ ]:
class Generator(nn.Module):
    # initializers
    def __init__(self,output_dim, noise_dim=32):
        super(Generator, self).__init__()
        input_out_feats = 64
        emb_dim = 23
#         output_dim = 38 + emb_dim
        
        self.input_layer = nn.Linear(in_features=noise_dim, out_features=input_out_feats)
        self.emb_y = nn.Sequential(nn.Embedding(num_embeddings=10,embedding_dim=emb_dim), nn.Flatten())
        self.model = nn.Sequential( nn.Linear(input_out_feats + emb_dim, 128), nn.ReLU(),  # 64 + 5
                                    nn.Linear(128, 256), nn.ReLU(),
                                    nn.Linear(256, 512), nn.ReLU(),
                                    nn.Linear(512, output_dim), nn.Tanh())
        
    def forward(self, x, y):
         x = self.input_layer(x)
#          print(x.shape)
         y = self.emb_y(y)
#          print(y.shape)
         return self.model(torch.cat((x, y), dim=1))
        
        
class Discriminator(nn.Module):
    def __init__(self,input_size):
        super(Discriminator, self).__init__()
        
        emb_dim = 23
        self.emb_y = nn.Sequential(nn.Embedding(num_embeddings=10,embedding_dim=emb_dim),nn.Flatten())
        self.model = nn.Sequential(nn.Linear(input_size + emb_dim, 512), nn.LeakyReLU(),
                                   nn.Linear(512, 256), nn.LeakyReLU(),
                                   nn.Linear(256, 128), nn.LeakyReLU(),
                                   nn.Linear(128, 1), nn.Sigmoid())

    def forward(self, x, y):
        y = self.emb_y(y)
        return self.model(torch.cat((x, y),dim=1))

In [ ]:
def gen_noise(BATCH_SIZE, z_noise):
    return torch.tensor(np.random.uniform(0., 1., size=[BATCH_SIZE, z_noise]))

gen_noise(64,32).shape

torch.Size([64, 32])

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

Assigning hyperparameters of the model and initilizing

In [ ]:
# label-smoothing
REAL = 0.9
FAKE = 0.1
BATCH_SIZE = 64
Z_DIM = 32
learning_rate = 0.0005
n_epochs = 10

D = Discriminator(input_size=41).to(device).float()
G = Generator(output_dim=41, noise_dim=Z_DIM).to(device).float()

loss_module = nn.BCELoss()

d_optimizer = optim.SGD(D.parameters(), learning_rate)
g_optimizer = optim.SGD(G.parameters(), learning_rate)

loss = nn.BCELoss()

Train loop

In [ ]:
from statistics import mean
dataloader = DataLoader(Loader(np.array(df), np.array(labels)), batch_size=64,shuffle=True,drop_last=True)

for epoch in range(n_epochs):
  D_loss = []
  G_loss = []
  G.train()
  D.train()
  for true_data, labels in dataloader:
    true_data = true_data.to(device).float()
    labels = labels.to(device)
    BATCH_SIZE = true_data.shape[0]

    #Train discriminator
    noise = gen_noise(64, 32).to(device)    
    fake_labels = torch.randint(0, 4, (BATCH_SIZE,)).to(device)
    generated_data = G(noise.float(), fake_labels) 
    true_labels = torch.ones(BATCH_SIZE).to(device)

    d_optimizer.zero_grad()
    discriminator_output_for_true_data = D(true_data, labels).view(BATCH_SIZE)
    true_discriminator_loss = loss(discriminator_output_for_true_data, true_labels)

    discriminator_output_for_generated_data = D(generated_data.detach(), fake_labels).view(BATCH_SIZE)
    generator_discriminator_loss = loss(discriminator_output_for_generated_data, torch.zeros(BATCH_SIZE).to(device))
    discriminator_loss = (true_discriminator_loss + generator_discriminator_loss)/2
            
    discriminator_loss.backward()
    d_optimizer.step()

    D_loss.append(discriminator_loss.data.item())

    g_optimizer.zero_grad()

    #Train generator
    generated_data = G(noise.float(), fake_labels) 
    discriminator_output_on_generated_data = D(generated_data, fake_labels).view(BATCH_SIZE)
    generator_loss = loss(discriminator_output_on_generated_data, true_labels)
    generator_loss.backward()
    g_optimizer.step()
    G_loss.append(generator_loss.data.item())
    
  print(mean(D_loss), mean(G_loss))

0.6765335171751055 0.7194955089789492
0.6307462523478842 0.7788777243877771
0.5159743801004132 0.972003840603677
0.3702584975563988 1.3718843624599661
0.32076118884838684 1.8507185262587438
0.2961478844072837 2.0856944726914026
0.27901490063682965 2.235911784445382
0.26038889141359567 2.253657038040342
0.27886250684756614 2.1755013528554357
0.2680949429747135 2.315219775348216


In [ ]:
torch.save(G,'Generator.pth')

Using trained model we generate new model:

we try to balance the data

In [ ]:
label,occ=np.unique(le.transform(names),return_counts=True)
label

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
maximum_repeat=label[occ == np.amax(occ)][0]
generated_data={}
for item in label:
  size = occ[maximum_repeat] - occ[item]
  y = torch.ones((size), dtype=torch.int64).to(device) * item
  z = np.random.uniform(-1, 1, size=(size, 32))
  z = torch.from_numpy(z).float().to(device)

  generated_data[item]= G(z, y).cpu().detach().numpy()

Adding newly generated data to previous data that we had

In [ ]:
new_X = []
new_y = []
for label, data in generated_data.items():
    size = len(data)
    new_y.append(np.ones(size) * label)
    new_X.append(data)
new_X.append(np.array(df))
new_y.append(np.array(classes).reshape(1,-1)[0])
new_X_train = np.concatenate(new_X)
new_y_train = np.concatenate(new_y)
# new_X_train = pd.concat([df,pd.DataFrame(new_X)],axis = 1)
# new_y_train = pd.concat([classes, pd.DataFrame(new_y)], axis = 1)

In [ ]:
y_label = np.around(new_X_train[:,-1])
new_X_train = np.concatenate((new_X_train,new_y_train.reshape(-1,1)),axis = 1)
new_X_train.shape , y_label.shape

((560000, 42), (560000,))

### 2.2 Classification models on newly generated data

In [ ]:
def split(data, label):
  X_train, X_test, y_train, y_test = train_test_split(
  data, label, test_size=0.2, random_state=42)
  return X_train, X_test, y_train, y_test


In [ ]:
X_train, X_test, y_train, y_test = split(new_X_train, y_label)

After spliting the data, we are using random forest classifier to see the quality of cGAN

Since the data became massive in balancing step, classification take some time to finish 

In [25]:
clf = RandomForestClassifier()
clf.fit(new_X_train,new_y_train)
y_pred=clf.predict(X_test)
print(classification_report(y_pred,y_test))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00     11243
         1.0       0.11      1.00      0.20     11141
         2.0       0.00      0.00      0.00     11061
         3.0       0.00      0.00      0.00     11268
         4.0       0.00      0.00      0.00     11211
         5.0       0.00      0.00      0.00     11182
         6.0       0.00      0.00      0.00     11388
         7.0       0.00      0.00      0.00     11117
         8.0       0.00      0.00      0.00     11098
         9.0       0.00      0.00      0.00     11291

    accuracy                           0.10    112000
   macro avg       0.01      0.10      0.02    112000
weighted avg       0.01      0.10      0.02    112000



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


and here also we use Explainable Boosting Classifier model

In [26]:
!pip install interpret

In [27]:
clf = ExplainableBoostingClassifier(max_bins=3, n_jobs=-1, interactions=0, max_leaves=25)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(classification_report(y_pred,y_test))


              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     11388
         1.0       1.00      1.00      1.00    100612

    accuracy                           1.00    112000
   macro avg       1.00      1.00      1.00    112000
weighted avg       1.00      1.00      1.00    112000



Final classifier is a simple Neural Network

In [ ]:
class Net(nn.Module):
    def __init__(self,inp,H,output):
        super(Net,self).__init__()
        self.linear1=nn.Linear(inp,H)
        self.linear2=nn.Linear(H,output)
        
    def forward(self,x):
        x=torch.sigmoid(self.linear1(x))  
        x=self.linear2(x)
        return x

    # def __init_weights(self,m):
    #   if type(m) == nn.Linear:
    #     torch.nn.init.xavier_uniform_(m.weight)
    #     m.bias.data.fill_(0.01)


input_dim,hidden_dim,output_dim = 42, 10, 1
SimpleNN = Net(input_dim,hidden_dim,output_dim)

Assigning hyperparameters:

In [ ]:
batch_size = 32
learning_rate = 0.002 
num_epochs = 10



train_dataset = TensorDataset(X_train, y_train)
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size,
                                              shuffle=True, num_workers=2)

test_dataset = TensorDataset(X_test, y_test)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size,
                                              shuffle=True, num_workers=2)

training model with X_train

In [ ]:
criterion = nn.MSE()
optimizer = torch.optim.Adam(SimpleNN.parameters(),lr=learning_rate)

for epoch in range(num_epochs): 
  epoch_loss = list() 
  for X_batch in train_dataset: 
    optimizer.zero_grad() 

    inputs, labels = X_batch
    inputs, labels = inputs.to(device), labels.to(device)
    
    output = SimpleNN(inputs) 
    loss = criterion(labels, output) 
 
    loss.backward() 
    optimizer.step() 
 
    epoch_loss.append(loss.item()) 
  print(f'[{epoch}/{num_epochs}]', f'loss: {np.array(epoch_loss).mean()}')


testing:

In [ ]:
with torch.no_grad():
  SimpleNN.eval()
  X_test, y_test = testloader
  y_pred = pred(SimpleNN(X_test)).to_numpy().detach()
  classification_report(np.around(y_pred),y_test)
